In [ ]:
!pip install tensorflow_text==2.8.1
import tensorflow as tf
print(tf.__version__)
import tensorflow_hub as hub
import tensorflow_text as text


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2.8.0


In [ ]:
print("No. of GPUS available:",len(tf.config.experimental.list_physical_devices('GPU')))

No. of GPUS available: 1


In [ ]:
import pandas as pd
df = pd.read_csv("combined_data.csv",names=['text','label'])
print(df.head(5))

print(df.groupby('label').describe())
print("After removing Duplicates:")
df.drop_duplicates(subset='text',keep='first',inplace=True)
df.groupby('label').describe()


                                                text  label
0  #Depressed mood can be caused by infectious di...      0
1  With all of this unnessary  family drama, I fe...      1
2  I'm going to keep banging on about this, cos i...      0
3  Ugh. Woke up to the same acne covered face thi...      0
4  #stoner #instahookah #depressed Portable Trans...      0
        text                                                               
       count unique                                                top freq
label                                                                      
0      11470   9985    haIZ....frOM laSt wEek i keEP on goiNg out.....    6
1       5709   4373  Mom's depression tied to kids' emotional, inte...   11
After removing Duplicates:


text                                                               
      count unique                                                top freq
label                                                                     
0      9985   9985  #Depressed mood can be caused by infectious di...    1
1      4371   4371  With all of this unnessary  family drama, I fe...    1

In [ ]:
df_positive=df[df['label']==1]
df_negative=df[df['label']==0]
df_negative_down = df_negative.sample(df_positive.shape[0])
df_negative_down.shape

df_balanced = pd.concat([df_positive,df_negative_down])
df_balanced

,text,label
1,"With all of this unnessary family drama, I fe...",1
6,"I feel so sad because i can't be happy, and th...",1
7,I went to church with my mom and a huge chunk ...,1
8,No love here.... #lonely #depressed pic.twitt...,1
9,I see so many people moving forward with their...,1
...,...,...
14152,@sekyw The writers are huddled over there with...,0
4651,Having troubles reconnecting with an old frie...,0
13276,@jordanknight I like her thinking,0
1663,The Fall of the Pure A tear falls from my e...,0


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_balanced['text'],df_balanced['label'],stratify=df_balanced['label'])

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4",trainable=True)

In [ ]:
def get_sentence_embedding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

get_sentence_embedding(["$110 discount, hurry up","Bhavin, are you up for a football game tonight?"])

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.83215076, -0.49884498, -0.83281636, ..., -0.6315708 ,
        -0.7261329 ,  0.9110628 ],
       [-0.8599927 , -0.50692946, -0.9453913 , ..., -0.8701028 ,
        -0.6732962 ,  0.8294471 ]], dtype=float32)>

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
"""#bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")
preprocessed = bert_preprocess(text_input)
embeddings = bert_encoder(preprocessed)
print(embeddings['pooled_output'])
#neural network layers
cls_embedding = tf.keras.layers.Dropout(0.1, name='dropout')(embeddings['pooled_output'])

l = tf.keras.layers.Dense(1,activation='sigmoid',name='output')(cls_embedding)

#l = tf.keras.layers.Dense(units=2,activation='softmax')(cls_embedding)
#l = tf.keras.layers.Dense(1,name='output')(l)

#construct final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])"""

'#bert layers\ntext_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")\npreprocessed = bert_preprocess(text_input)\nembeddings = bert_encoder(preprocessed)\nprint(embeddings[\'pooled_output\'])\n#neural network layers\ncls_embedding = tf.keras.layers.Dropout(0.1, name=\'dropout\')(embeddings[\'pooled_output\'])\n\nl = tf.keras.layers.Dense(1,activation=\'sigmoid\',name=\'output\')(cls_embedding)\n\n#l = tf.keras.layers.Dense(units=2,activation=\'softmax\')(cls_embedding)\n#l = tf.keras.layers.Dense(1,name=\'output\')(l)\n\n#construct final model\nmodel = tf.keras.Model(inputs=[text_input], outputs = [l])'

In [ ]:
#bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")
preprocessed = bert_preprocess(text_input)
embeddings = bert_encoder(preprocessed)
print(embeddings['pooled_output'])
#neural network layers
cls_embedding = tf.keras.layers.Dropout(0.2, name='dropout')(embeddings['pooled_output'])

#l = tf.keras.layers.Dense(1,activation='sigmoid',name='output')(cls_embedding)

X = tf.keras.layers.BatchNormalization()(cls_embedding)
X = tf.keras.layers.Dense(192, activation='relu')(X)
#X = tf.keras.layers.Dropout(0.2)(X)
X = tf.keras.layers.Dense(1, activation='softmax')(X)


#construct final model
model = tf.keras.Model(inputs=[text_input], outputs = X)

KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='keras_layer_1/StatefulPartitionedCall:13', description="created by layer 'keras_layer_1'")


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [ ]:
METRICS = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
]
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=METRICS)

In [ ]:
model.fit(X_train,y_train,epochs=10)

Epoch 1/10
205/205 [==============================] - 191s 820ms/step - loss: 0.7840 - accuracy: 0.5000 - precision: 0.5000 - recall: 1.0000
Epoch 2/10
205/205 [==============================] - 175s 854ms/step - loss: 0.7409 - accuracy: 0.5000 - precision: 0.5000 - recall: 1.0000
Epoch 3/10
205/205 [==============================] - 177s 863ms/step - loss: 0.7125 - accuracy: 0.5000 - precision: 0.5000 - recall: 1.0000
Epoch 4/10
205/205 [==============================] - 177s 862ms/step - loss: 0.7092 - accuracy: 0.5000 - precision: 0.5000 - recall: 1.0000
Epoch 5/10
205/205 [==============================] - 177s 864ms/step - loss: 0.7028 - accuracy: 0.5000 - precision: 0.5000 - recall: 1.0000
Epoch 6/10
205/205 [==============================] - 177s 863ms/step - loss: 0.6967 - accuracy: 0.5000 - precision: 0.5000 - recall: 1.0000
Epoch 7/10
205/205 [==============================] - 177s 863ms/step - loss: 0.6982 - accuracy: 0.5000 - precision: 0.5000 - recall: 1.0000
Epoch 8/10
20

In [ ]:
model.evaluate(X_test,y_test)

69/69 [==============================] - 23s 321ms/step - loss: 0.6931 - accuracy: 0.5000 - precision: 0.5000 - recall: 1.0000


[0.6931483149528503, 0.5, 0.5, 1.0]

In [ ]:
"""y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()
y_predicted = np.where(y_predicted > 0.5 , 1, 0)
from sklearn.metrics import classification_report
print(classification_report(y_test,y_predicted))"""

'y_predicted = model.predict(X_test)\ny_predicted = y_predicted.flatten()\ny_predicted = np.where(y_predicted > 0.5 , 1, 0)\nfrom sklearn.metrics import classification_report\nprint(classification_report(y_test,y_predicted))'